# Introduction

Since Jan. 1, 2015, [The Washington Post](https://www.washingtonpost.com/) has been compiling a database of every fatal shooting in the US by a police officer in the line of duty. 

<center><img src=https://i.imgur.com/sX3K62b.png></center>

While there are many challenges regarding data collection and reporting, The Washington Post has been tracking more than a dozen details about each killing. This includes the race, age and gender of the deceased, whether the person was armed, and whether the victim was experiencing a mental-health crisis. The Washington Post has gathered this supplemental information from law enforcement websites, local new reports, social media, and by monitoring independent databases such as "Killed by police" and "Fatal Encounters". The Post has also conducted additional reporting in many cases.

There are 4 additional datasets: US census data on poverty rate, high school graduation rate, median household income, and racial demographics. [Source of census data](https://factfinder.census.gov/faces/nav/jsf/pages/community_facts.xhtml).

### Upgrade Plotly

Run the cell below if you are working with Google Colab

In [1]:
%pip install --upgrade plotly

Note: you may need to restart the kernel to use updated packages.


c:\Users\spiacentini\Documents\GitHub\Professional Portfolio\Analyze Deaths involving Police in the US\.venv\Scripts\python.exe: No module named pip


## Import Statements

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# This might be helpful:
from collections import Counter


# Add kernel with uv add --dev ipykernel

## Notebook Presentation

In [11]:
pd.options.display.float_format = '{:,.2f}'.format

## Load the Data

In [4]:
df_hh_income = pd.read_csv('Median_Household_Income_2015.csv', encoding="windows-1252")
df_pct_poverty = pd.read_csv('Pct_People_Below_Poverty_Level.csv', encoding="windows-1252")
df_pct_completed_hs = pd.read_csv('Pct_Over_25_Completed_High_School.csv', encoding="windows-1252")
df_share_race_city = pd.read_csv('Share_of_Race_By_City.csv', encoding="windows-1252")
df_fatalities = pd.read_csv('Deaths_by_Police_US.csv', encoding="windows-1252")

# Preliminary Data Exploration

* What is the shape of the DataFrames? 
* How many rows and columns do they have?
* What are the column names?
* Are there any NaN values or duplicates?

In [5]:
dfs = {
    'df_hh_income':df_hh_income,
    'df_pct_poverty':df_pct_poverty,
    'df_pct_completed_hs':df_pct_completed_hs,
    'df_share_race_city':df_share_race_city,
    'df_fatalities':df_fatalities
}

for name,df in dfs.items():
    print(f"The shape of {name}: {df.shape}")
    print(f"There are {df.shape[0]} rows and {df.shape[1]} columns")
    print(f"Those column names are {df.columns}")
    print(f"Are there NaNs? {df.isna().any().any()}")
    print(f"Are there duplicates? {df.duplicated().any()}")

The shape of df_hh_income: (29322, 3)
There are 29322 rows and 3 columns
Those column names are Index(['Geographic Area', 'City', 'Median Income'], dtype='object')
Are there NaNs? True
Are there duplicates? False
The shape of df_pct_poverty: (29329, 3)
There are 29329 rows and 3 columns
Those column names are Index(['Geographic Area', 'City', 'poverty_rate'], dtype='object')
Are there NaNs? False
Are there duplicates? False
The shape of df_pct_completed_hs: (29329, 3)
There are 29329 rows and 3 columns
Those column names are Index(['Geographic Area', 'City', 'percent_completed_hs'], dtype='object')
Are there NaNs? False
Are there duplicates? False
The shape of df_share_race_city: (29268, 7)
There are 29268 rows and 7 columns
Those column names are Index(['Geographic area', 'City', 'share_white', 'share_black',
       'share_native_american', 'share_asian', 'share_hispanic'],
      dtype='object')
Are there NaNs? False
Are there duplicates? False
The shape of df_fatalities: (2535, 14)
T

## Data Cleaning - Check for Missing Values and Duplicates

Consider how to deal with the NaN values. Perhaps substituting 0 is appropriate. 

In [6]:
# Clean up the nan with 
for name,df in dfs.items():
    df.fillna(0)

# Chart the Poverty Rate in each US State

Create a bar chart that ranks the poverty rate from highest to lowest by US state. Which state has the highest poverty rate? Which state has the lowest poverty rate?  Bar Plot

In [7]:
df_pct_poverty.head()

,Geographic Area,City,poverty_rate
0,AL,Abanda CDP,78.8
1,AL,Abbeville city,29.1
2,AL,Adamsville city,25.5
3,AL,Addison town,30.7
4,AL,Akron town,42


In [85]:
df_pct_poverty['poverty_rate'] = pd.to_numeric(df_pct_poverty['poverty_rate'],errors='coerce')

df_pov_state = df_pct_poverty.copy()
total = df_pov_state.groupby('Geographic Area').size()

sum_rate= df_pov_state.groupby('Geographic Area')['poverty_rate'].sum()


series = sum_rate/total


df_by_state = series.reset_index(name='Pct Pov')
df_by_state.sort_values('Pct Pov',ascending=False,inplace=True)

fig = px.bar(df_by_state,x='Geographic Area',y='Pct Pov',title='Percent Poverty By State',color='Geographic Area')
fig.show()

# Chart the High School Graduation Rate by US State

Show the High School Graduation Rate in ascending order of US States. Which state has the lowest high school graduation rate? Which state has the highest?

In [86]:
df_pct_completed_hs.head()

,Geographic Area,City,percent_completed_hs
0,AL,Abanda CDP,21.2
1,AL,Abbeville city,69.1
2,AL,Adamsville city,78.9
3,AL,Addison town,81.4
4,AL,Akron town,68.6


In [95]:
df_pct_completed_hs['percent_completed_hs'] = pd.to_numeric(df_pct_completed_hs['percent_completed_hs'],errors='coerce')

total = df_pct_completed_hs.groupby('Geographic Area').size()

grad_rates = df_pct_completed_hs.groupby('Geographic Area')['percent_completed_hs'].sum()

grad_pct = grad_rates/total

df_grad_pct_state = grad_pct.reset_index(name='Grad Pct')

df_grad_pct_state.sort_values('Grad Pct',ascending=True,inplace=True)

fig = px.bar(df_grad_pct_state,x='Geographic Area',y='Grad Pct',color='Geographic Area',title='Graduation Rate by State in ASC Order')
fig.show()

# Visualise the Relationship between Poverty Rates and High School Graduation Rates

#### Create a line chart with two y-axes to show if the rations of poverty and high school graduation move together.  

#### Now use a Seaborn .jointplot() with a Kernel Density Estimate (KDE) and/or scatter plot to visualise the same relationship

#### Seaborn's `.lmplot()` or `.regplot()` to show a linear regression between the poverty ratio and the high school graduation ratio. 

# Create a Bar Chart with Subsections Showing the Racial Makeup of Each US State

Visualise the share of the white, black, hispanic, asian and native american population in each US State using a bar chart with sub sections. 

# Create Donut Chart by of People Killed by Race

Hint: Use `.value_counts()`

# Create a Chart Comparing the Total Number of Deaths of Men and Women

Use `df_fatalities` to illustrate how many more men are killed compared to women. 

# Create a Box Plot Showing the Age and Manner of Death

Break out the data by gender using `df_fatalities`. Is there a difference between men and women in the manner of death? 

# Were People Armed? 

In what percentage of police killings were people armed? Create chart that show what kind of weapon (if any) the deceased was carrying. How many of the people killed by police were armed with guns versus unarmed? 

# How Old Were the People Killed?

Work out what percentage of people killed were under 25 years old.  

Create a histogram and KDE plot that shows the distribution of ages of the people killed by police. 

Create a seperate KDE plot for each race. Is there a difference between the distributions? 

# Race of People Killed

Create a chart that shows the total number of people killed by race. 

# Mental Illness and Police Killings

What percentage of people killed by police have been diagnosed with a mental illness?

# In Which Cities Do the Most Police Killings Take Place?

Create a chart ranking the top 10 cities with the most police killings. Which cities are the most dangerous?  

# Rate of Death by Race

Find the share of each race in the top 10 cities. Contrast this with the top 10 cities of police killings to work out the rate at which people are killed by race for each city. 

# Create a Choropleth Map of Police Killings by US State

Which states are the most dangerous? Compare your map with your previous chart. Are these the same states with high degrees of poverty? 

# Number of Police Killings Over Time

Analyse the Number of Police Killings over Time. Is there a trend in the data? 

# Epilogue

Now that you have analysed the data yourself, read [The Washington Post's analysis here](https://www.washingtonpost.com/graphics/investigations/police-shootings-database/).